In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 1000
# mpl.rcParams['legend.fontsize'] = 15
# params = {'legend.fontsize': 20,
#           'legend.handlelength': 2}
# plot.rcParams.update(params)

In [3]:
from algorithms.moo.nsga2 import NSGA2
from constraints.as_obj import ConstraintsAsObjective
from operators.survival.rank_and_crowding.classes import (
    RankAndCrowding,
    MyConstrRankAndCrowding,
    ParallelConstrRankAndCrowding,
    MyConstrRankAndCrowding2,
)
from optimize import minimize
from visualization.scatter import Scatter
from pymoo.problems import get_problem

In [4]:
core_moea1 = NSGA2(pop_size=500, survival=RankAndCrowding())
core_moea2 = NSGA2(pop_size=500, survival=MyConstrRankAndCrowding())
core_moea3 = NSGA2(pop_size=500, survival=ParallelConstrRankAndCrowding())
core_moea4 = NSGA2(pop_size=500, survival=MyConstrRankAndCrowding2())

In [5]:
import time


def performance_timer(func):
    """
    A decorator to measure and print the execution time of a function.
    """

    def wrapper(*args, **kwargs):
        start_time = time.time()  # Capture the start time
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # Capture the end time
        duration = end_time - start_time  # Calculate duration
        print(f"{func.__name__} executed in {duration:.6f} seconds")
        return result

    return wrapper


@performance_timer
def minimize_(*args, **kwargs):
    return minimize(*args, **kwargs)

In [6]:
from pymoo.indicators.gd import GD
from pymoo.indicators.gd_plus import GDPlus
from pymoo.indicators.igd import IGD
from pymoo.indicators.igd_plus import IGDPlus
from pymoo.indicators.hv import HV

metric_classes = {
    "Generational Distance (GD)": GD,
    "Generational Distance Plus (GD+)": GDPlus,
    "Inverted Generational Distance (IGD)": IGD,
    "Inverted Generational Distance Plus (IGD+)": IGDPlus,
    # "Hypervolume": HV,
}


def performance_indicators(pf, results):
    # Pre-configuration for each metric, handle special cases like HV here.
    pf_calculators = [
        (
            metric_class(ref_point=np.array([0, 0]))
            if metric_name == "Hypervolume"
            else metric_class(pf)
        )
        for metric_name, metric_class in metric_classes.items()
    ]

    def cal_acc_metric(sols):
        results_matrix = np.empty((len(sols), len(metric_classes)), dtype=object)

        for idx, sol in enumerate(sols):
            # Ensure `sol` is a 2D array with the correct shape [n_samples, n_features]
            sol = np.atleast_2d(sol)
            if sol.shape[0] == 1 and sol.shape[1] != len(pf):
                # This implies sol is a single sample with multiple features, which is the correct format
                pass
            elif sol.shape[1] == 1 and sol.shape[0] != len(pf):
                # This implies sol has the wrong orientation; it's many samples of a single feature
                sol = sol.T

            for metric_idx, calculator in enumerate(pf_calculators):
                try:
                    results_matrix[idx, metric_idx] = calculator(sol)
                except Exception as e:
                    # print(
                    #     f"Error calculating {list(metric_classes.keys())[metric_idx]}: {e}")
                    results_matrix[idx, metric_idx] = np.nan

        return [
            (np.mean(results_matrix[:, i]), np.median(results_matrix[:, i]))
            for i in range(len(pf_calculators))
        ]

    return [cal_acc_metric(rs) for rs in results]

In [7]:
from constraints.as_obj import CVRAsObjective, CDFAsObjective, CVRAsObjective2, CDFAsObjective2, CDFAsObjective3


@performance_timer
def compare_on(problem, n_gen=300):
    # Configuration for each optimization strategy
    configurations = [
        (problem, core_moea1, ("n_gen", n_gen), {"seed": 1, "verbose": False}),
        # (ConstraintsAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CDFAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (ConstraintsAsObjective(problem),
        #  core_moea2, ("n_gen", n_gen), {"seed": 1}),
        # (
        #     CVRAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        (
            CDFAsObjective2(problem),
            core_moea4,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
        # (
        #     CDFAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        (
            CDFAsObjective3(problem),
            core_moea4,
            ("n_gen", n_gen),
            {"seed": 1, "verbose": False},
        ),
    ]

    results = []
    for config in configurations:
        result = minimize_(*config[:-1], **config[-1])
        results.append(result)
    return results


def test(problem_name, n_gen=300, *args):
    if "dascmop" in problem_name:
        print(problem_name, args)
        problem = get_problem(problem_name, args[0])
    elif "modact" in problem_name:
        problem = get_problem(problem_name, args[0])
    else:
        print(problem_name)
        problem = get_problem(problem_name)
    pf = problem.pareto_front()
    print(
        f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
    )

    results = compare_on(problem, n_gen=n_gen)

    # Check if any of the results is None
    if any(result is None for result in results):
        print("One or more optimization runs failed to return a result.")
        # Return None in place of results to indicate failure
        return pf, [None] * len(results)

    # Prepare data for performance indicators, safely handling cases where result.F might be None or incorrectly shaped
    prepared_data = []
    for i, result in enumerate(results):
        if result and hasattr(result, "F") and result.F is not None:
            # i == 0 this NGSA without additional objective
            if i > 0:
                # if i == 1 or i == 3:
                # if True:
                prepared_data.append(result.F[:, :-2])
            else:
                # prepared_data.append(result.F[:, :-1])
                prepared_data.append(result.F)
        else:
            # Use None or an appropriate placeholder if result is invalid
            print(f'algo {i} not converge')
            prepared_data.append(None)

    # Ensure all data is valid before calculating performance indicators
    if all(data is not None for data in prepared_data):
        performance_results = performance_indicators(pf, prepared_data)
        for i, metric in enumerate(metric_classes.keys()):
            print(
                f"{metric}: " +
                " | ".join(f"{perf[i]}" for perf in performance_results)
            )
    else:
        print("Performance indicators could not be calculated due to invalid data.")

    return pf, *results

# DASCMOP 1

In [ ]:
pf, *results = test("dascmop1", 500, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop2", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop3", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop4", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop5", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop6", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop7", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop8", 300, 1)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop9", 300, 1)
pf, results[0].F, results[1].F, results[2].F

# DASCMOP 2

In [ ]:
pf, *results = test("dascmop1", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop2", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop3", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop4", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop5", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop5", 300, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop6", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop7", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop8", 300, 2)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop9", 300, 2)
pf, results[0].F, results[1].F, results[2].F

# DASCMOP 3

In [ ]:
pf, *results = test("dascmop1", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop2", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop3", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop4", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop5", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop6", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop7", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop8", 500, 3)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("dascmop9", 500, 3)
pf, results[0].F, results[1].F, results[2].F

# DASCMOP 4

In [24]:
pf, *results = test("dascmop1", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119d8de50>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 34.647352 seconds
minimize_ executed in 30.171866 seconds
minimize_ executed in 30.044990 seconds
compare_on executed in 94.864948 seconds
Generational Distance (GD): (0.0003556870398065451, 0.0003554176179200717) | (0.0003579743074405028, 0.0003444641655278656) | (0.00034812388835791933, 0.0003256877335808028)
Generational Distance Plus (GD+): (0.00029198747596411925, 0.0002799906927150042) | (0.0002900854884590197, 0.00026870796495309657) | (0.0002807635853839286, 0.00025676681560410364)
Inverted Generational Distance (IGD): (0.4425297730157356, 0.43496386321756514) | (0.43307403767741837, 0.42918004851758507) | (0.4395670863299907, 0.43245141745921706)
Inverted Generational Distance Plus (IGD+): (0.3293304309873457, 0.3284982460777453) | (0.3282264283899155, 0.32761878212774176) | (0.32899107019412493

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.82368842, 1.39529967],
        [0.74692949, 1.43906925],
        [0.80382652, 1.40781566],
        [0.74550419, 1.43988573],
        [0.75001437, 1.43761223],
        [0.80602822, 1.40638545],
        [0.80843942, 1.40487725],
        [0.79193214, 1.415157  ],
        [0.72732755, 1.44851607],
        [0.79215206, 1.41465991],
        [0.74905307, 1.43812833],
        [0.79361304, 1.41385945],
        [0.74471344, 1.44019461],
        [0.79511555, 1.41308978],
        [0.73221449, 1.4461434 ],
        [0.794954  , 1.41318829],
        [0.79928604, 1.41054598],
        [0.79738653, 1.41156187],
        [0.73474609, 1.44496178],
        [0.72671957, 1.44871128],
        [0.7395262 , 1.44269144],
        [0.74397747, 1.44048009],
        [0.74455936, 1.44025507],
        [0.80559257, 1.

In [25]:
pf, *results = test("dascmop2", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x11a6519a0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.476127 seconds
minimize_ executed in 29.748017 seconds
minimize_ executed in 30.263876 seconds
compare_on executed in 93.488575 seconds
Generational Distance (GD): (0.0003907825023690712, 0.0003822074690869387) | (0.00041433106134750275, 0.00037626307575962833) | (0.0003978590917318327, 0.0004056628630249879)
Generational Distance Plus (GD+): (0.00033937505639242637, 0.000327626074743792) | (0.00036281503582300174, 0.00031497557025217393) | (0.00034957209801279014, 0.00033521228601307573)
Inverted Generational Distance (IGD): (0.36080758737658525, 0.3579284715455353) | (0.3626220953774955, 0.35931400272391933) | (0.3520308105407455, 0.3522414921330544)
Inverted Generational Distance Plus (IGD+): (0.21330070204995755, 0.21054397582190243) | (0.21418766663295521, 0.21134220673223547) | (0.20840481987481

(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.82193805, 0.93269189],
        [0.82111692, 0.93333544],
        [0.84282604, 0.91460219],
        [0.74432975, 1.00604532],
        [0.84405823, 0.91361988],
        [0.82141557, 0.93311718],
        [0.82186972, 0.93273613],
        [0.82367777, 0.93107821],
        [0.83441345, 0.92175445],
        [0.82202611, 0.93253114],
        [0.74712849, 1.00329167],
        [0.80898098, 0.94416875],
        [0.83065444, 0.92530826],
        [0.84253461, 0.91476044],
        [0.82283635, 0.93196206],
        [0.83069553, 0.92497711],
        [0.80708441, 0.94591544],
        [0.85021777, 0.90849539],
        [0.80551979, 0.9473101 ],
        [0.82931684, 0.92619943],
        [0.79622922, 0.95580605],
        [0.80497492, 0.9478087 ],
        [0.89779607, 0.86961042],

In [26]:
pf, *results = test("dascmop3", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x11a698910>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.692409 seconds
minimize_ executed in 30.991493 seconds
minimize_ executed in 29.769148 seconds
compare_on executed in 93.453860 seconds
Generational Distance (GD): (0.14749805621133733, 0.195742789797537) | (0.002456412505818402, 0.002442360060581364) | (0.1508091135767639, 0.18991898117440656)
Generational Distance Plus (GD+): (0.13552565968478766, 0.18301405199217669) | (0.0007492158295479954, 0.0005015003242083704) | (0.14078130415255707, 0.18131982579385075)
Inverted Generational Distance (IGD): (0.49779456467722333, 0.5141635203917957) | (0.4123550517313609, 0.41160792283581593) | (0.5065482920527009, 0.5195582984333046)
Inverted Generational Distance Plus (IGD+): (0.3865317016539016, 0.42957255037884323) | (0.22138007087705477, 0.22013351719558472) | (0.40056210046208307, 0.43689978426464404)


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.69219219, 1.1227715 ],
        [0.69319319, 1.1138213 ],
        [0.69419419, 1.1048608 ],
        [0.6951952 , 1.095892  ],
        [0.6961962 , 1.0869168 ],
        [0.6971972 , 1.0779371 ],
        [0.6981982 , 1.0689549 ],
        [0.6991992 , 1.0599719 ],
        [0.7002002 , 1.054135  ],
        [0.89239239, 0.93319559],
        [0.89339339, 0.92458428],
        [0.89439439, 0.91596118],
        [0.8953954 , 0.90732822],
        [0.8963964 , 0.89868734],
        [0.8973974 , 0.89004046],
        [0.898

In [27]:
pf, *results = test("dascmop4", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x119c25e50>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.640995 seconds
minimize_ executed in 24.266936 seconds
minimize_ executed in 22.811781 seconds
compare_on executed in 79.720427 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[1.32013275, 0.82749288],
        [1.32228793, 0.82469966],
        [1.19261958, 1.02031199],
        [1.4579167 , 0.58265437],
        [1.40620835, 0.68117578],
        [1.3270827 , 0.81661407],
        [1.24486104, 0.94632481],
        [1.15051754, 1.07706285],
        [1.23889553, 0.95432695],
        [1.43983227, 0.61715279],
        [1.24443039, 0.9468597 ],
        [1.21382917, 0.99182266],
        [1.33561447, 0.80187134],
        [1.14707925, 1.08217466],
        [1.31085301, 0.84258051],
        [1.42665859, 0.64296661],
        [1.31914614, 0.83051055],
        [1.33221088, 0.80932906],
        [1.31077739, 0.84308614],
        [0.55277865, 1.49725839],
        [1.45447108, 0.58941248],
        [1.20957401, 0.99682791],
        [1.24801244, 0.94067465],
        [1.30590126, 0.

In [28]:
pf, *results = test("dascmop5", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x119c36f70>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.958086 seconds
minimize_ executed in 24.765480 seconds
minimize_ executed in 24.442363 seconds
compare_on executed in 85.166665 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.81605688, 0.93817452],
        [0.71757931, 1.0353853 ],
        [0.7325918 , 1.01773514],
        [0.62093677, 1.15512748],
        [1.09888062, 0.72746233],
        [0.51446077, 1.38249678],
        [1.32595607, 0.59153504],
        [0.61792703, 1.15877649],
        [0.91145232, 0.85918812],
        [1.3577476 , 0.57423165],
        [0.5428924 , 1.2933312 ],
        [1.2378334 , 0.641081  ],
        [0.53790504, 1.30732526],
        [0.91764394, 0.85679979],
        [0.55396959, 1.27068638],
        [1.21490297, 0.6547969 ],
        [0.92655703, 0.84704145],
        [0.5282065 , 1.33478065],
        [0.71427234, 1.03898824],
        [0.74075235, 1.01008636],
        [0.84877837, 0.91052099],
        [0.9929298 , 0.79795302],
        [0.8114681 , 0.9422875 ],

In [29]:
pf, *results = test("dascmop6", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x11a122340>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 32.767791 seconds
minimize_ executed in 25.026153 seconds
minimize_ executed in 27.124003 seconds
compare_on executed in 84.919179 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.69219219, 1.1227715 ],
        [0.69319319, 1.1138213 ],
        [0.69419419, 1.1048608 ],
        [0.6951952 , 1.095892  ],
        [0.6961962 , 1.0869168 ],
        [0.6971972 , 1.0779371 ],
        [0.6981982 , 1.0689549 ],
        [0.6991992 , 1.0599719 ],
        [0.7002002 , 1.054135  ],
        [0.89239239, 0.93319559],
        [0.89339339, 0.92458428],
        [0.89439439, 0.91596118],
        [0.8953954 , 0.90732822],
        [0.8963964 , 0.89868734],
        [0.8973974 , 0.89004046],
        [0.898

In [30]:
pf, *results = test("dascmop7", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x11a9ad040>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 34.226772 seconds
minimize_ executed in 23.581834 seconds
minimize_ executed in 23.792029 seconds
compare_on executed in 81.601328 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 0.5       , 1.5       ],
        [0.5       , 0.51010101, 1.489899  ],
        [0.5       , 0.52020202, 1.479798  ],
        ...,
        [1.479798  , 0.5       , 0.52020202],
        [1.489899  , 0.5       , 0.51010101],
        [1.5       , 0.5       , 0.5       ]]),
 array([[1.50039356, 0.50092633, 0.50092844],
        [0.5084436 , 1.49115514, 0.50370317],
        [0.5006303 , 0.50042204, 1.49925631],
        ...,
        [1.01449404, 0.77306032, 0.73199854],
        [0.78565106, 0.53269147, 1.18850363],
        [1.39384024, 0.51780097, 0.62916951]]),
 None,
 None)

In [31]:
pf, *results = test("dascmop8", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x119f2d640>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 34.089945 seconds
minimize_ executed in 23.562363 seconds
minimize_ executed in 24.916681 seconds
compare_on executed in 82.569708 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[1.48436969, 0.50049077, 0.67716515],
        [0.50024865, 0.50301973, 1.50010395],
        [0.50089881, 0.50212124, 1.50083996],
        ...,
        [1.32955392, 0.94214651, 0.85304459],
        [1.288518  , 0.52450613, 1.12219674],
        [1.10671492, 1.13326465, 0.98923511]]),
 None,
 None)

In [32]:
pf, *results = test("dascmop9", 500, 4)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (4,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x119e31760>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 35.708602 seconds
minimize_ executed in 31.253175 seconds
minimize_ executed in 29.505310 seconds
compare_on executed in 96.467670 seconds
Generational Distance (GD): (0.005574046901285079, 0.005479457711344652) | (0.005770016191012888, 0.005779715342435808) | (0.005338712211496662, 0.005287300002004903)
Generational Distance Plus (GD+): (0.004168716115726188, 0.0037054250370208708) | (0.004148928027218507, 0.0038048168716435724) | (0.003900027573151151, 0.0034205024955416352)
Inverted Generational Distance (IGD): (0.6164842109748915, 0.6028470570174316) | (0.6020992421018433, 0.5940743137244371) | (0.6135523895233471, 0.6085076367606523)
Inverted Generational Distance Plus (IGD+): (0.4332924476242349, 0.4239439598474637) | (0.4273283609823944, 0.4161261378982421) | (0.4322371646148525, 0.4242898285725467

(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.90685857, 1.26700782, 1.02693882],
        [0.50017568, 0.79486979, 1.45561192],
        [0.51498783, 1.50023306, 0.56841598],
        ...,
        [0.68076357, 1.05988533, 1.30900121],
        [0.76282439, 1.08314373, 1.26882685],
        [0.55077727, 0.87392498, 1.42709143]]),
 array([[0.50125417, 0.50149643, 1.50057371, 0.        , 0.        ],
        [0.52483911, 1.46164914, 0.8610207 , 0.        , 0.        ],
        [0.50004014, 0.87720437, 1.42613823, 0.        , 0.        ],
        ...,
        [0.60570296, 1.13824257, 1.27150275, 0.        , 0.        ],
        [0.51381368, 1.21538507, 1.19928854, 0.        , 0.        ],
        [0.6130531 , 1.14637534, 1.262868  , 0.        

# DASCMOP 5

In [33]:
pf, *results = test("dascmop1", 500, 5)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (5,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119c36fd0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 36.517284 seconds
minimize_ executed in 32.791672 seconds
minimize_ executed in 31.768495 seconds
compare_on executed in 101.089874 seconds
Generational Distance (GD): (0.00040630323368679973, 0.00035839312963238115) | (0.00040370469828418607, 0.0003541731425896735) | (0.00040370469828418607, 0.0003541731425896735)
Generational Distance Plus (GD+): (0.00035412855660320926, 0.00029495834062776005) | (0.00035767967306190093, 0.00030160322271786293) | (0.00035767967306190093, 0.00030160322271786293)
Inverted Generational Distance (IGD): (0.37887136502331, 0.3786087211556472) | (0.3777657689059739, 0.3778949491979246) | (0.3777657689059739, 0.3778949491979246)
Inverted Generational Distance Plus (IGD+): (0.29077320432848247, 0.290612826853629) | (0.29056527227397017, 0.2905040070792242) | (0.2905652722739701

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.94894895, 0.09949589],
        [0.94994995, 0.09759509],
        [1.        , 0.        ]]),
 array([[0.31229173, 0.90854135],
        [0.34546338, 0.88400713],
        [0.31235356, 0.90805491],
        [0.31441377, 0.90342272],
        [0.34130956, 0.88545613],
        [0.33834047, 0.8864571 ],
        [0.34206032, 0.88514317],
        [0.34466361, 0.88490575],
        [0.33766447, 0.88676216],
        [0.34473334, 0.88421974],
        [0.31478246, 0.90295492],
        [0.31528273, 0.90234438],
        [0.3155877 , 0.90154396],
        [0.31737758, 0.90024299],
        [0.31528822, 0.90220865],
        [0.33709256, 0.88711948],
        [0.31740562, 0.89980649],
        [0.31770808, 0.89954914],
        [0.31451074, 0.90319465],
        [0.33650856, 0.88757823],
        [0.33466293, 0.88830951],
        [0.31676064, 0.90041602],
        [0.33271447, 0.88955206],

In [34]:
pf, *results = test("dascmop1", 500, 6)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (6,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x10b9aeb20>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.348212 seconds
minimize_ executed in 30.766906 seconds
minimize_ executed in 31.897888 seconds
compare_on executed in 98.013560 seconds
Generational Distance (GD): (0.0003896841963297798, 0.0003660162474315054) | (0.00039630092812079765, 0.0003877615993521002) | (0.00039630092812079765, 0.0003877615993521002)
Generational Distance Plus (GD+): (0.0003436169548438371, 0.00031160396543650215) | (0.0003438083050930045, 0.00032342125578383655) | (0.0003438083050930045, 0.00032342125578383655)
Inverted Generational Distance (IGD): (0.4007171600595103, 0.39839878657496036) | (0.406754702319969, 0.4054359067601392) | (0.406754702319969, 0.4054359067601392)
Inverted Generational Distance Plus (IGD+): (0.3045271265126768, 0.30406342699716954) | (0.30619167265471, 0.30600391915749914) | (0.30619167265471, 0.3060

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.82953996, 1.39200682],
        [0.82310006, 1.3956856 ],
        [0.82165498, 1.39675812],
        [0.76571193, 1.42954177],
        [0.83360121, 1.38878219],
        [0.83196167, 1.39005703],
        [0.78830616, 1.41693776],
        [0.82672966, 1.39346242],
        [0.82584002, 1.39399947],
        [0.82724111, 1.39293837],
        [0.8165356 , 1.39992853],
        [0.80817432, 1.40506922],
        [0.80371403, 1.40776704],
        [0.82316677, 1.39558229],
        [0.85853433, 1.37145655],
        [0.80414674, 1.40767796],
        [0.84649698, 1.3799657 ],
        [0.80791702, 1.40518846],
        [0.86519771, 1.36677358],
        [0.81127467, 1.40318649],
        [0.86785547, 1.36473581],
        [0.76976434, 1.4273776 ],
        [0.82659873, 1.3937196 ],
        [0.80950001, 1.

In [35]:
pf, *results = test("dascmop1", 500, 7)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (7,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119d69fa0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 39.950913 seconds
minimize_ executed in 37.439197 seconds
minimize_ executed in 32.563504 seconds
compare_on executed in 109.954319 seconds
Generational Distance (GD): (0.0003800550138108663, 0.0003619658297349381) | (0.0003800550138108663, 0.0003619658297349381) | (0.0003800550138108663, 0.0003619658297349381)
Generational Distance Plus (GD+): (0.00032380424525261554, 0.00028713387635317977) | (0.00032380424525261554, 0.00028713387635317977) | (0.00032380424525261554, 0.00028713387635317977)
Inverted Generational Distance (IGD): (0.3245513374081239, 0.3244878411933325) | (0.3245513374081239, 0.3244878411933325) | (0.3245513374081239, 0.3244878411933325)
Inverted Generational Distance Plus (IGD+): (0.25593667017600263, 0.2558815893433345) | (0.25593667017600263, 0.2558815893433345) | (0.25593667017600263

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[0.31269809, 0.90584266],
        [0.3398424 , 0.88806085],
        [0.33559833, 0.88874878],
        [0.31314333, 0.90463814],
        [0.31405886, 0.90331276],
        [0.31477582, 0.90290099],
        [0.31489423, 0.9017274 ],
        [0.31300866, 0.90532706],
        [0.33391962, 0.88920532],
        [0.33326747, 0.8896403 ],
        [0.33462292, 0.8890536 ],
        [0.31406774, 0.90300084],
        [0.33157035, 0.89048044],
        [0.33293965, 0.88974039],
        [0.33193709, 0.89026695],
        [0.31568891, 0.90139784],
        [0.31599228, 0.90091497],
        [0.31544551, 0.90155788],
        [0.31639655, 0.9006883 ],
        [0.33253558, 0.88993137],
        [0.33500745, 0.8888202 ],
        [0.3321865 , 0.89013339],
        [0.3166139 , 0.90024822],

In [36]:
pf, *results = test("dascmop1", 500, 8)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (8,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119c36f70>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 31.512660 seconds
minimize_ executed in 25.867455 seconds
minimize_ executed in 26.081847 seconds
compare_on executed in 83.462790 seconds
Generational Distance (GD): (0.5600611483121528, 0.5634174054907533) | (0.5600835651813842, 0.563806684037034) | (0.5199454105949315, 0.5411255613865769)
Generational Distance Plus (GD+): (0.5600611483121528, 0.5634174054907533) | (0.5600835651813842, 0.563806684037034) | (0.5199454105949315, 0.5411255613865769)
Inverted Generational Distance (IGD): (0.8082535333716115, 0.8091580128596527) | (0.8091852206642888, 0.8092480492710861) | (0.7507871851330429, 0.7603849031200192)
Inverted Generational Distance Plus (IGD+): (0.797502653000016, 0.7988869372106394) | (0.7982721456521185, 0.7989710385865045) | (0.7479267067049414, 0.7573013021662824)


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        [0.503003  , 1.499991  ],
        [0.504004  , 1.499984  ],
        [0.50500501, 1.4999749 ],
        [0.50600601, 1.4999639 ],
        [0.50700701, 1.4999509 ],
        [0.50800801, 1.4999359 ],
        [0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.529

In [37]:
pf, *results = test("dascmop1", 500, 9)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (9,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x10315c430>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 35.032852 seconds
minimize_ executed in 31.090128 seconds
minimize_ executed in 31.083876 seconds
compare_on executed in 97.207509 seconds
Generational Distance (GD): (0.0003483672401223666, 0.0003344445891038013) | (0.000310784500237706, 0.0003002944962222713) | (0.000310784500237706, 0.0003002944962222713)
Generational Distance Plus (GD+): (0.00029022193151945495, 0.0002708946148059358) | (0.00023982505675096074, 0.00022930018475919578) | (0.00023982505675096074, 0.00022930018475919578)
Inverted Generational Distance (IGD): (0.37823876512978216, 0.3782512588636907) | (0.37996350221748754, 0.3800419554192179) | (0.37996350221748754, 0.3800419554192179)
Inverted Generational Distance Plus (IGD+): (0.2879562051060003, 0.2878921715248107) | (0.28836480965122946, 0.28835350020760975) | (0.28836480965122946,

(array([[0.00900901, 0.99991884],
        [0.01001001, 0.9998998 ],
        [0.01101101, 0.99987876],
        [0.01201201, 0.99985571],
        [0.01301301, 0.99983066],
        [0.01401401, 0.99980361],
        [0.01501501, 0.99977455],
        [0.01601602, 0.99974349],
        [0.01701702, 0.99971042],
        [0.01801802, 0.99967535],
        [0.01901902, 0.99963828],
        [0.02002002, 0.9995992 ],
        [0.02102102, 0.99955812],
        [0.02202202, 0.99951503],
        [0.02302302, 0.99946994],
        [0.02402402, 0.99942285],
        [0.02502502, 0.99937375],
        [0.02602603, 0.99932265],
        [0.02702703, 0.99926954],
        [0.02802803, 0.99921443],
        [0.02902903, 0.99915732],
        [0.03003003, 0.9990982 ],
        [0.03103103, 0.99903708],
        [0.03203203, 0.99897395],
        [0.03303303, 0.99890882],
        [0.03403403, 0.99884168],
        [0.03503503, 0.99877255],
        [0.03603604, 0.9987014 ],
        [0.03703704, 0.99862826],
        [0.038

In [38]:
pf, *results = test("dascmop1", 500, 10)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (10,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x103232f70>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.342900 seconds
minimize_ executed in 27.974256 seconds
minimize_ executed in 28.594375 seconds
compare_on executed in 89.912210 seconds
Generational Distance (GD): (0.0003668361806386893, 0.000353331191273738) | (0.00038124035217707177, 0.00035041616446419756) | (0.00038124035217707177, 0.00035041616446419756)
Generational Distance Plus (GD+): (0.00031977563903037013, 0.0003026516031533452) | (0.00033521090396641513, 0.00030207887747108776) | (0.00033521090396641513, 0.00030207887747108776)
Inverted Generational Distance (IGD): (0.3989891011757778, 0.3981220344260037) | (0.40416811246327816, 0.4043247649375712) | (0.40416811246327816, 0.4043247649375712)
Inverted Generational Distance Plus (IGD+): (0.3040531493323313, 0.30390986045019347) | (0.30545613621283596, 0.30561734232523213) | (0.305456136212

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.81989364, 1.39806691],
        [0.78260406, 1.42023652],
        [0.75088644, 1.43749587],
        [0.80712336, 1.40567764],
        [0.84510475, 1.38113351],
        [0.87179706, 1.36218456],
        [0.88047853, 1.35613735],
        [0.87603014, 1.35874972],
        [0.7899442 , 1.41594411],
        [0.82879737, 1.39196467],
        [0.8143627 , 1.4015697 ],
        [0.81567128, 1.40074217],
        [0.84028673, 1.38433884],
        [0.84025523, 1.38444594],
        [0.81095944, 1.40367   ],
        [0.79225314, 1.41489043],
        [0.87374663, 1.36050215],
        [0.80305545, 1.4082581 ],
        [0.81770773, 1.39912528],
        [0.83263562, 1.38946162],
        [0.88131683, 1.35465385],
        [0.81629513, 1.40004701],
        [0.87223308, 1.36159085],
        [0.79892957, 1.

In [39]:
pf, *results = test("dascmop1", 500, 11)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (11,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119f9a760>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.364827 seconds
minimize_ executed in 34.234980 seconds
minimize_ executed in 33.831180 seconds
compare_on executed in 101.431636 seconds
Generational Distance (GD): (0.00042496216300023373, 0.0003540382959134587) | (0.00042496216300023373, 0.0003540382959134587) | (0.00042496216300023373, 0.0003540382959134587)
Generational Distance Plus (GD+): (0.0003782309211442391, 0.00030629608181123623) | (0.0003782309211442391, 0.00030629608181123623) | (0.0003782309211442391, 0.00030629608181123623)
Inverted Generational Distance (IGD): (0.2606770604750384, 0.26055633040213627) | (0.2606770604750384, 0.26055633040213627) | (0.2606770604750384, 0.26055633040213627)
Inverted Generational Distance Plus (IGD+): (0.21857219798106986, 0.21842021151143104) | (0.21857219798106986, 0.21842021151143104) | (0.21857219798

(array([[0.        , 1.        ],
        [0.001001  , 0.999999  ],
        [0.002002  , 0.99999599],
        ...,
        [0.997998  , 0.004     ],
        [0.998999  , 0.002001  ],
        [1.        , 0.        ]]),
 array([[0.3373735 , 0.8925723 ],
        [0.36969294, 0.86789149],
        [0.33784429, 0.89000452],
        [0.36545098, 0.86853781],
        [0.36846128, 0.86816135],
        [0.33770582, 0.89212255],
        [0.3385381 , 0.88801234],
        [0.3635183 , 0.86901768],
        [0.34072193, 0.88460457],
        [0.33930371, 0.88709698],
        [0.33960815, 0.8860998 ],
        [0.33909215, 0.8873181 ],
        [0.34040589, 0.88546198],
        [0.36928766, 0.86790046],
        [0.33867668, 0.88790528],
        [0.33979266, 0.88564452],
        [0.34119529, 0.88443763],
        [0.36316543, 0.86932386],
        [0.36272964, 0.86959575],
        [0.34028433, 0.88549474],
        [0.34159657, 0.88390984],
        [0.34274301, 0.88288501],
        [0.36179534, 0.869886  ],

In [40]:
pf, *results = test("dascmop1", 500, 12)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (12,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x11a1f95b0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 27.132771 seconds
minimize_ executed in 23.089835 seconds
minimize_ executed in 23.193703 seconds
compare_on executed in 73.416962 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.52902903, 1.4991573 ],
        [0.53003003, 1.4990982 ],
        [0.53103103, 1.4990371 ],
        [0.53203203, 1.4989739 ],
        [0.53303303, 1.4989088 ],
        [0.53403403, 1.4988417 ],
        [0.53503504, 1.4987725 ],
        [0.53603604, 1.4987014 ],
        [0.53703704, 1.4986283 ],
        [0.538

In [41]:
pf, *results = test("dascmop1", 500, 13)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (13,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119f9ac10>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 31.712538 seconds
minimize_ executed in 27.344030 seconds
minimize_ executed in 27.310081 seconds
compare_on executed in 86.367239 seconds
Generational Distance (GD): (0.0003309622272543898, 0.0003150111685871954) | (0.000315029564214432, 0.0002854283441396768) | (0.000315029564214432, 0.0002854283441396768)
Generational Distance Plus (GD+): (0.00011802329532818723, 0.00010793147684384774) | (0.0001453763355318385, 0.00013840108657159345) | (0.0001453763355318385, 0.00013840108657159345)
Inverted Generational Distance (IGD): (0.3999804562132814, 0.4001673965516553) | (0.407039433668768, 0.4080257785476228) | (0.407039433668768, 0.4080257785476228)
Inverted Generational Distance Plus (IGD+): (0.3042876940456254, 0.30433830101123005) | (0.3062172556757226, 0.3065065845760472) | (0.3062172556757226, 0.3065

(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.79617381, 1.41220291],
        [0.88270992, 1.3534863 ],
        [0.85071822, 1.3769737 ],
        [0.84406951, 1.38165104],
        [0.83601455, 1.38724174],
        [0.84121557, 1.38345419],
        [0.79755591, 1.41133244],
        [0.80182829, 1.40876581],
        [0.80385164, 1.4075347 ],
        [0.83430509, 1.38817875],
        [0.80753397, 1.40526402],
        [0.81076296, 1.40332473],
        [0.83206889, 1.38959373],
        [0.84223901, 1.38279134],
        [0.80911602, 1.40430839],
        [0.80312089, 1.40795858],
        [0.81196318, 1.40258444],
        [0.79641436, 1.41208151],
        [0.83185059, 1.39000628],
        [0.80238375, 1.40842392],
        [0.82617136, 1.39359753],
        [0.82726378, 1.3930131 ],
        [0.82332472, 1.39533696],
        [0.82243077, 1.

In [42]:
pf, *results = test("dascmop1", 500, 14)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (14,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119f74a60>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 30.294413 seconds
minimize_ executed in 26.253803 seconds
minimize_ executed in 23.901078 seconds
compare_on executed in 80.449888 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        ...,
        [1.4489489 , 0.59949589],
        [1.4499499 , 0.59759509],
        [1.5       , 0.5       ]]),
 array([[0.80326541, 1.40798804],
        [0.84974092, 1.37752145],
        [0.82781248, 1.39265459],
        [0.8130302 , 1.40185229],
        [0.832564  , 1.38929085],
        [0.8090455 , 1.40440869],
        [0.84233511, 1.38271409],
        [0.84541422, 1.38076883],
        [0.83735148, 1.3860858 ],
        [0.83910145, 1.38497921],
        [0.82655553, 1.39320728],
        [0.80492018, 1.40699516],
        [0.80665779, 1.40584719],
        [0.81755757, 1.39907956],
        [0.84722595, 1.37942611],
        [0.83633755, 1.38684707],
        [0.84947484, 1.37778032],
        [0.80559494, 1.40658233],
        [0.84115052, 1.38356855],
        [0.81430516, 1.40111698],
        [0.84833082, 1.37867345],
        [0.81566063, 1.40019844],
        [0.81473426, 1.40080643],

# DASCMOP 15


In [8]:
pf, *results = test("dascmop1", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x103232f70>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.179307 seconds
minimize_ executed in 31.147283 seconds
minimize_ executed in 32.719563 seconds
compare_on executed in 97.047182 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 None,
 None,
 None)

In [9]:
pf, *results = test("dascmop2", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x119c25130>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 39.288834 seconds
minimize_ executed in 30.269850 seconds
minimize_ executed in 32.449662 seconds
compare_on executed in 102.009149 seconds
algo 1 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.74914354, 1.00066114],
        [0.8220699 , 0.93234376],
        [0.80524738, 0.94734038],
        [0.83562018, 0.92050585],
        [0.81427976, 0.93929127],
        [0.82605326, 0.92891971],
        [0.83648605, 0.92006818],
        [0.82799377, 0.92713741],
        [0.81613879, 0.9376667 ],
        [0.8157869 , 0.93797334],
        [0.81539938, 0.93844684],
        [0.81581528, 0.93783713],
        [0.82681112, 0.92819026],
        [0.83679388, 0.91952799],
        [0.8360977 , 0.92020675],
        [0.83610062, 0.9201119 ],
        [0.83204043, 0.92364954],
        [0.80793036, 0.94495219],
        [0.80635107, 0.94639005],
        [0.82826789, 0.92694384],
        [0.80524061, 0.94734324],
        [0.81627023, 0.9376062 ],
        [0.81515935, 0.93851847],

In [10]:
pf, *results = test("dascmop3", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x119c253d0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 40.415371 seconds
minimize_ executed in 31.393746 seconds
minimize_ executed in 31.973133 seconds
compare_on executed in 103.783185 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

In [11]:
pf, *results = test("dascmop3", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x103232fa0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 39.122719 seconds
minimize_ executed in 29.314199 seconds
minimize_ executed in 29.500430 seconds
compare_on executed in 97.938238 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

In [12]:
pf, *results = test("dascmop4", 500, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x119da1b50>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 33.861164 seconds
minimize_ executed in 33.793307 seconds
minimize_ executed in 59.407762 seconds
compare_on executed in 127.063089 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5     , 1.5     ],
        [0.501001, 1.499999],
        [0.502002, 1.499996],
        ...,
        [1.497998, 0.504   ],
        [1.498999, 0.502001],
        [1.5     , 0.5     ]]),
 array([[0.50538852, 1.49990456],
        [1.49985756, 0.50000391],
        [0.65793661, 1.47503824],
        [1.14240347, 1.08708975],
        [0.53670896, 1.49868462],
        [0.647593  , 1.47822048],
        [0.52150064, 1.4995766 ],
        [0.56211632, 1.49614721],
        [0.58537105, 1.49268606],
        [0.55313595, 1.49721781],
        [0.57627618, 1.49424675],
        [0.58835027, 1.49213993],
        [0.56750043, 1.49533543],
        [0.51124475, 1.49980844],
        [0.54955075, 1.49749007],
        [0.6183992 , 1.48587675],
        [0.62842231, 1.4835989 ],
        [0.60343307, 1.48926499],
        [0.59576551, 1.49088223],
        [0.60891911, 1.48823173],
        [0.61408716, 1.48701307],
        [0.51590803, 1.49972752],
        [0.54355328, 1.49815948],
        [1.31322087, 0.

In [13]:
pf, *results = test("dascmop5", 1000, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x11a09e8b0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 84.183724 seconds
minimize_ executed in 63.694227 seconds
minimize_ executed in 61.920848 seconds
compare_on executed in 209.799677 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.497998  , 0.5010015 ],
        [1.498999  , 0.50050063],
        [1.5       , 0.5       ]]),
 array([[0.8382888 , 0.91837373],
        [0.49990839, 1.49700402],
        [0.52146856, 1.35304921],
        [0.61118405, 1.16632563],
        [0.50007177, 1.48735911],
        [0.50421805, 1.43454129],
        [0.82418319, 0.93044939],
        [0.51982078, 1.35882955],
        [0.80876957, 0.94425543],
        [0.53326338, 1.31729973],
        [0.51894483, 1.36195862],
        [0.52235397, 1.35014471],
        [0.52289019, 1.34836774],
        [0.72927108, 1.0210589 ],
        [0.67366776, 1.08305214],
        [0.58669236, 1.20541874],
        [0.75594993, 0.99395197],
        [0.65228379, 1.10954804],
        [0.58806205, 1.20298031],
        [0.74637903, 1.00345677],
        [0.55312528, 1.26921595],
        [0.53421883, 1.31469084],
        [0.50467169, 1.43129833],

In [14]:
pf, *results = test("dascmop6", 1000, 15)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (15,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x119dcc2b0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 79.284591 seconds
minimize_ executed in 63.944319 seconds
minimize_ executed in 61.615271 seconds
compare_on executed in 204.844988 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.65115115, 1.4583213 ],
        [0.65215215, 1.4513346 ],
        [0.65315315, 1.4442677 ],
        [0.65415415, 1.4371221 ],
        [0.65515516, 1.429899  ],
        [0.65615616, 1.4226001 ],
        [0.65715716, 1.4152266 ],
        [0.65815816, 1.40778   ],
        [0.65915916, 1.400262  ],
        [0.66016016, 1.3926739 ],
        [0.66116116, 1.3850173 ],
        [0.66216216, 1.3772938 ],
        [0.66316316, 1.3695048 ],
        [0.66416416, 1.3616521 ],
        [0.66516517, 1.3537372 ],
        [0.66616617, 1.3457617 ],
        [0.66716717, 1.3377272 ],
        [0.66816817, 1.3296355 ],
        [0.66916917, 1.3214881 ],
        [0.67017017, 1.3132867 ],
        [0.67117117, 1.3050331 ],
        [0.67217217, 1.296729  ],
        [0.67317317, 1.288376  ],
        [0.674

# DASCMOP 16

In [15]:
pf, *results = test("dascmop1", 1000, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop1 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP1 object at 0x119c25dc0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 73.787047 seconds
minimize_ executed in 60.862576 seconds
minimize_ executed in 63.441949 seconds
compare_on executed in 198.092436 seconds
algo 0 not converge
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        [0.503003  , 1.499991  ],
        [0.504004  , 1.499984  ],
        [0.50500501, 1.4999749 ],
        [0.50600601, 1.4999639 ],
        [0.50700701, 1.4999509 ],
        [0.50800801, 1.4999359 ],
        [0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.529

In [16]:
pf, *results = test("dascmop2", 1000, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop2 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP2 object at 0x103232fa0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 82.216598 seconds
minimize_ executed in 58.590618 seconds
minimize_ executed in 58.902150 seconds
compare_on executed in 199.710202 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        ...,
        [1.4489489 , 0.52585989],
        [1.4499499 , 0.52534624],
        [1.5       , 0.5       ]]),
 array([[0.80864642, 0.94428463],
        [0.82814518, 0.92700855],
        [0.81390342, 0.93954726],
        [0.83508306, 0.92110544],
        [0.8155588 , 0.93827923],
        [0.83030367, 0.92509569],
        [0.8212149 , 0.93323071],
        [0.82645884, 0.92846286],
        [0.80680101, 0.94610867],
        [0.83659187, 0.91968006],
        [0.82090113, 0.93334572],
        [0.82091777, 0.93334084],
        [0.82520157, 0.92964446],
        [0.821722  , 0.93262115],
        [0.82205504, 0.93233202],
        [0.82829551, 0.92688541],
        [0.82041849, 0.93376374],
        [0.81530521, 0.93831883],
        [0.82169368, 0.93263139],
        [0.82077858, 0.93346462],
        [0.80790159, 0.94493727],
        [0.82809372, 0.92701894],
        [0.82735739, 0.92766786],

In [17]:
pf, *results = test("dascmop3", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop3 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP3 object at 0x10b9b9160>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 40.535752 seconds
minimize_ executed in 2018.066673 seconds
minimize_ executed in 32.867294 seconds
compare_on executed in 2091.470380 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.7002002 , 1.054135  ],
        [0.9044044 , 0.89863654],
        [1.1066066 , 0.77294554],
        [1.3008008 , 0.61141457],
        [1.5       , 0.5       ]]),
 array([[0.9040743 , 0.89641814],
        [0.8494158 , 1.26493356]]),
 None,
 None)

In [18]:
pf, *results = test("dascmop4", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop4 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP4 object at 0x11a9ad580>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 40.370246 seconds
minimize_ executed in 32.020860 seconds
minimize_ executed in 31.959299 seconds
compare_on executed in 104.351194 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.499999  ],
        [0.502002  , 1.499996  ],
        [0.503003  , 1.499991  ],
        [0.504004  , 1.499984  ],
        [0.50500501, 1.4999749 ],
        [0.50600601, 1.4999639 ],
        [0.50700701, 1.4999509 ],
        [0.50800801, 1.4999359 ],
        [0.50900901, 1.4999188 ],
        [0.51001001, 1.4998998 ],
        [0.51101101, 1.4998788 ],
        [0.51201201, 1.4998557 ],
        [0.51301301, 1.4998307 ],
        [0.51401401, 1.4998036 ],
        [0.51501502, 1.4997745 ],
        [0.51601602, 1.4997435 ],
        [0.51701702, 1.4997104 ],
        [0.51801802, 1.4996754 ],
        [0.51901902, 1.4996383 ],
        [0.52002002, 1.4995992 ],
        [0.52102102, 1.4995581 ],
        [0.52202202, 1.499515  ],
        [0.52302302, 1.4994699 ],
        [0.52402402, 1.4994228 ],
        [0.52502503, 1.4993737 ],
        [0.52602603, 1.4993226 ],
        [0.52702703, 1.4992695 ],
        [0.52802803, 1.4992144 ],
        [0.529

In [19]:
pf, *results = test("dascmop5", 200, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop5 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP5 object at 0x11aa384c0>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 14.056500 seconds
minimize_ executed in 12.183263 seconds
minimize_ executed in 9.134263 seconds
compare_on executed in 35.374711 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4683614 ],
        [0.502002  , 1.4552563 ],
        [0.503003  , 1.4452003 ],
        [0.504004  , 1.4367228 ],
        [0.50500501, 1.4292539 ],
        [0.50600601, 1.4225016 ],
        [0.50700701, 1.4162921 ],
        [0.50800801, 1.4105125 ],
        [0.50900901, 1.4050842 ],
        [0.51001001, 1.39995   ],
        [0.51101101, 1.3950666 ],
        [0.51201201, 1.3904007 ],
        [0.51301301, 1.3859254 ],
        [0.51401401, 1.3816192 ],
        [0.51501502, 1.3774642 ],
        [0.51601602, 1.3734456 ],
        [0.51701702, 1.3695507 ],
        [0.51801802, 1.3657688 ],
        [0.51901902, 1.3620905 ],
        [0.52002002, 1.3585079 ],
        [0.52102102, 1.3550137 ],
        [0.52202202, 1.3516018 ],
        [0.52302302, 1.3482666 ],
        [0.52402402, 1.3450031 ],
        [0.52502503, 1.341807  ],
        [0.52602603, 1.3386742 ],
        [0.52702703, 1.335601  ],
        [0.52802803, 1.3325843 ],
        [0.529

In [20]:
pf, *results = test("dascmop6", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop6 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP6 object at 0x119c25a00>>: n_var=30 n_obj=2 n_ieq_constr=11 n_eq_constr=0
minimize_ executed in 30.723689 seconds
minimize_ executed in 25.487853 seconds
minimize_ executed in 26.547340 seconds
compare_on executed in 82.759430 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 1.5       ],
        [0.501001  , 1.4762229 ],
        [0.502002  , 1.4709774 ],
        [0.503003  , 1.4687771 ],
        [0.504004  , 1.4681494 ],
        [0.65015015, 1.4652264 ],
        [0.7002002 , 1.054135  ],
        [0.9044044 , 0.89863654],
        [1.1066066 , 0.77294554],
        [1.3008008 , 0.61141457],
        [1.5       , 0.5       ]]),
 array([[0.50249704, 1.46939181],
        [0.50246097, 1.46943513],
        [0.50197684, 1.47071033],
        [0.50047859, 1.48049675],
        [0.50028156, 1.48349278],
        [0.50017583, 1.48550352],
        [0.5030877 , 1.4685128 ],
        [0.50262504, 1.46911926],
        [0.50081018, 1.47693578],
        [0.50291611, 1.46873881],
        [0.50335242, 1.4682471 ],
        [0.50030533, 1.48310168],
        [0.50224867, 1.4701623 ],
        [0.50042427, 1.48113279],
        [0.50112606, 1.47464899],
        [0.50005139, 1.4896619 ],
        [0.50309296, 1.46847289],
        [0.50231341, 1.46976372],
        [0.5

In [21]:
pf, *results = test("dascmop7", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop7 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP7 object at 0x11a8f16d0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 30.159043 seconds
minimize_ executed in 28.045642 seconds
minimize_ executed in 26.362021 seconds
compare_on executed in 84.567311 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[0.5       , 0.5       , 1.5       ],
        [0.5       , 0.51010101, 1.489899  ],
        [0.5       , 0.52020202, 1.479798  ],
        ...,
        [1.479798  , 0.5       , 0.52020202],
        [1.489899  , 0.5       , 0.51010101],
        [1.5       , 0.5       , 0.5       ]]),
 array([[1.00243994, 0.99730523, 0.49996263],
        [0.49990629, 0.49990599, 1.49989084],
        [0.50829701, 1.49065854, 0.50078379],
        ...,
        [0.73980967, 1.03569515, 0.72445104],
        [0.90964233, 0.90471573, 0.68535991],
        [1.21022701, 0.66649696, 0.62317305]]),
 None,
 None)

In [22]:
pf, *results = test("dascmop8", 500, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop8 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP8 object at 0x10b9d8be0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 31.130302 seconds
minimize_ executed in 24.231930 seconds
minimize_ executed in 25.896994 seconds
compare_on executed in 81.259868 seconds
algo 1 not converge
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.49999503, 1.49998145, 0.50082588],
        [0.54780704, 0.56227266, 1.496909  ],
        [1.49989982, 0.50016097, 0.50187337],
        ...,
        [0.66499068, 1.46068753, 0.7232386 ],
        [1.44581647, 0.534451  , 0.82250671],
        [1.30849879, 1.08807686, 0.51470635]]),
 None,
 None)

In [23]:
pf, *results = test("dascmop9", 200, 16)
pf, results[0].F, results[1].F, results[2].F

dascmop9 (16,)
Problem <bound method Problem.name of <pymoo.problems.multi.dascmop.DASCMOP9 object at 0x119fcaac0>>: n_var=30 n_obj=3 n_ieq_constr=7 n_eq_constr=0
minimize_ executed in 12.088988 seconds
minimize_ executed in 10.378944 seconds
minimize_ executed in 9.302902 seconds
compare_on executed in 31.771583 seconds
algo 2 not converge
Performance indicators could not be calculated due to invalid data.


(array([[1.5       , 0.5       , 0.5       ],
        [1.4998741 , 0.51586596, 0.5       ],
        [1.4994965 , 0.53172793, 0.5       ],
        ...,
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ],
        [0.5       , 0.5       , 1.5       ]]),
 array([[0.54789907, 0.80337469, 1.45165181],
        [0.53673427, 0.60448135, 1.49378019],
        [0.55207257, 0.92922831, 1.40154131],
        [0.55106387, 0.75970233, 1.46445316],
        [0.5797886 , 0.92151174, 1.4031889 ],
        [0.5706621 , 0.60345733, 1.49205112],
        [0.55486002, 0.79123637, 1.45510142],
        [0.54784103, 0.78240899, 1.45803723],
        [0.54665205, 0.78359409, 1.45769313],
        [0.52748518, 0.64461408, 1.48921021],
        [0.56226309, 0.59227588, 1.49370301],
        [0.54881871, 0.76651855, 1.46261582],
        [0.5843784 , 0.94337569, 1.39226734],
        [0.54768415, 0.78246138, 1.45805934],
        [0.55059229, 0.78278635, 1.45784178],
        [0.54796773